In [1]:
import json
import math as m

In [2]:
# seperate dataset by class 
def get_data_group_by_classes(dataset):
    seperated_category_dict = {category:[] for category in categories}
    for record in dataset:
        seperated_category_dict[record['category']].append(record)
    return seperated_category_dict

In [3]:
#checks if smoothing is needed
def is_smoothing_needed(text,word_dict,category):
    for word in text.split(" "):
        if word not in word_dict[category].keys():
            return True
    return False

# get probability of the text assuming conditional independence(naive bayes)         
def get_naive_probability(category,text,word_dict,grouped_classes_dict,dataset):
    text = text.lower()
    if category not in word_dict.keys():
        return
    total_word_cond_prob = 1
    smoothing_indicator = is_smoothing_needed(text,word_dict,category)
    for word in text.split(" "):
        if not smoothing_indicator:
            total_word_cond_prob *= (word_dict[category][word]/len(word_dict[category]))
        else:
            total_word_cond_prob *= (1/(len(word_dict[category])+len(word_dict[category]))) # Smoothing technique
    class_probability = (len(grouped_classes_dict[category])/len(dataset))
    return total_word_cond_prob * class_probability
       

In [4]:
#get the predicted class given text belong to
def get_predicted_class(categories,str_to_predict,word_frequency_dict,group_classes_dict,train_dataset):
    greatest_prob = 0
    predicted_category = ''
    for category in categories:
        total_word_cond_prob = 1
        prob_of_category_contains_headline =  get_naive_probability(category,str_to_predict,word_frequency_dict,group_classes_dict,train_dataset)
        if prob_of_category_contains_headline > greatest_prob:
            greatest_prob = prob_of_category_contains_headline
            predicted_category = category
        #print(f'''prob_of_category_contains_headline {category} is {prob_of_category_contains_headline}''')
    return predicted_category

In [5]:
# Load the original dataset. 
dataset = []
categories = []
with open('data/News_Category_Dataset_v2.json') as data:
    for line_num,record in enumerate(data,start=0):
        dataset.append(json.loads(record))
        if dataset[line_num]['category'] not in categories:
            categories.append(dataset[line_num]['category'])
no_of_records = len(dataset)
print(f""" Number of records in the dataset: {no_of_records}""")
print(f""" Number of different categories in the dataset: {len(categories)}""")


 Number of records in the dataset: 200853
 Number of different categories in the dataset: 41


In [6]:
#Divide the dataset as train, development and test.
train_split = m.ceil(no_of_records * 0.65)
dev_split = m.ceil(no_of_records * 0.20)
test_split =m.ceil(no_of_records * 0.15)
train_dataset = dataset[0:train_split]
dev_dataset = dataset[train_split+1:train_split+dev_split]
test_dataset = dataset[train_split+dev_split+1:]


In [7]:
# Build a vocab list 
vocab_list = []
for record in train_dataset:
    vocab_list.extend(record['headline'].split(" "))
vocab_list = [word.lower() for word in list(set(vocab_list))]
print(f''' Length of vocab list in training dataset: {len(vocab_list)}''')


 Length of vocab list in training dataset: 88261


In [8]:
#create count frequency of words on training data
group_classes_dict = get_data_group_by_classes(train_dataset)
word_frequency_dict = {category:{vocab:0 for vocab in vocab_list} for category in categories}
for record in train_dataset:
    for word in record['headline'].split(" "):
        word_frequency_dict[record['category']][word.lower()]+=1


In [ ]:
 #calculate the accuracy of the classifier over given dataset:
correct_predicted = 0
for record in train_dataset: 
    predicted_category = get_predicted_class(categories,record['headline'],word_frequency_dict,group_classes_dict,train_dataset)
    if predicted_category == record['category']:
        correct_predicted+=1
print(f'''Accuracy of the Naive Baye's Classifier over train dataset is {round((correct_predicted/len(train_dataset))*100,2)}%''')
